# ⛓️ **03 - Chains: Especialistas por Domínio**

## 🎯 **Objetivo:**
Criar 4 chains especializadas, cada uma expert em um domínio específico do turismo.

## 📋 **O que faremos:**
1. 🗺️ **Chain Roteiros**: Especialista em pontos turísticos
2. 🚗 **Chain Logística**: Expert em transporte e hospedagem
3. 📍 **Chain Info Local**: Conhecedor de cultura local
4. 🌐 **Chain Tradução**: Tradutor especializado

---

## 1️⃣ **Setup e Imports**

In [14]:
# Imports necessários
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
# ✅ Removido: from langchain.chains import LLMChain (depreciado)
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurar LLM
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
if not GROQ_API_KEY:
    print("⚠️ Configure GROQ_API_KEY no arquivo .env")
else:
    print("✅ Credenciais carregadas!")

# Inicializar LLM
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.1-8b-instant",
    temperature=0.1
)

print("🧠 LLM Groq inicializado com sintaxe moderna!")

✅ Credenciais carregadas!
🧠 LLM Groq inicializado com sintaxe moderna!


## 2️⃣ **Importar Sistema RAG**

In [15]:
def carregar_base_conhecimento_chains(arquivo_path: str = "base_conhecimento.txt") -> dict:
    """
    Carrega a base de conhecimento organizada para uso com chains.
    
    Returns:
        Dicionário organizado por especialidade e cidade
    """
    try:
        with open(arquivo_path, 'r', encoding='utf-8') as file:
            conteudo = file.read()
        
        dados = {
            "roteiro": {"rio": [], "paris": []},
            "logistica": {"rio": [], "paris": []}
        }
        
        linhas = conteudo.split('\n')
        secao_atual = None
        
        for linha in linhas:
            linha = linha.strip()
            if not linha or linha.startswith('#'):
                continue
                
            # Identificar seções
            if 'ROTEIROS - RIO' in linha:
                secao_atual = ("roteiro", "rio")
            elif 'LOGÍSTICA - RIO' in linha:
                secao_atual = ("logistica", "rio")
            elif 'ROTEIROS - PARIS' in linha:
                secao_atual = ("roteiro", "paris")
            elif 'LOGÍSTICA - PARIS' in linha:
                secao_atual = ("logistica", "paris")
            elif secao_atual and ':' in linha:
                # Adicionar linha de dados à seção atual
                tipo, cidade = secao_atual
                dados[tipo][cidade].append(linha)
        
        return dados
        
    except FileNotFoundError:
        print("⚠️ Arquivo base_conhecimento.txt não encontrado. Usando dados padrão.")
        return {
            "roteiro": {
                "rio": ["Cristo Redentor: Atração famosa do Rio."],
                "paris": ["Torre Eiffel: Símbolo de Paris."]
            },
            "logistica": {
                "rio": ["Metro Rio: Transporte público eficiente."],
                "paris": ["Metro Paris: Sistema integrado de transporte."]
            }
        }

def obter_contexto_rag(query: str, especialidade: str) -> str:
    """
    Obtém contexto relevante da base de conhecimento externa.
    Simula o sistema RAG usando dados do arquivo base_conhecimento.txt
    """
    # Carregar contextos da base de conhecimento
    contextos = carregar_base_conhecimento_chains()
    
    # Detectar cidade na query
    cidade = 'rio' if any(palavra in query.lower() for palavra in ['rio', 'brasil', 'copacabana', 'ipanema']) else 'paris'
    
    # Mapear especialidade para tipo de dados
    tipo_map = {
        'roteiro': 'roteiro',
        'logistica': 'logistica',
        'info-local': 'roteiro',  # Fallback para roteiro
        'traducao': 'roteiro'     # Fallback para roteiro
    }
    
    tipo_dados = tipo_map.get(especialidade, 'roteiro')
    
    # Obter contexto relevante
    if tipo_dados in contextos and cidade in contextos[tipo_dados]:
        infos = contextos[tipo_dados][cidade]
        if infos:
            return "\\n".join([f"• {info}" for info in infos])
    
    return "Informações não disponíveis na base de conhecimento."

print("✅ **SISTEMA RAG COM BASE DE CONHECIMENTO EXTERNA!**")
print("   📁 Fonte: base_conhecimento.txt")
print("   🔄 Carregamento dinâmico dos dados")
print("   🛡️ Fallback automático se arquivo não encontrado")

✅ **SISTEMA RAG COM BASE DE CONHECIMENTO EXTERNA!**
   📁 Fonte: base_conhecimento.txt
   🔄 Carregamento dinâmico dos dados
   🛡️ Fallback automático se arquivo não encontrado


## 3️⃣ **Chain 1: Especialista em Roteiros**

In [16]:
# Template para Chain de Roteiros
template_roteiro = """
🗺️ ESPECIALISTA EM ROTEIROS TURÍSTICOS

Você é um guia especializado em criar roteiros turísticos personalizados.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Pontos turísticos imperdíveis
- Roteiros otimizados por tempo
- Atrações por interesse (cultura, história, natureza)
- Dicas de horários e melhores épocas
- Tempo necessário para cada atração

INSTRUÇÕES:
- Use APENAS informações do contexto fornecido
- Seja específico sobre localização e características
- Sugira sequência lógica de visitação
- Inclua dicas práticas (horários, ingressos)
- Mantenha tom empolgante e informativo

RESPOSTA:
"""

# Criar PromptTemplate
prompt_roteiro = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_roteiro
)

# ✅ SINTAXE MODERNA: Usar RunnableSequence
chain_roteiro = prompt_roteiro | llm

print("🗺️ Chain de Roteiros criada com sintaxe moderna!")

🗺️ Chain de Roteiros criada com sintaxe moderna!



### **Teste da Chain Roteiros**

In [17]:
# Teste da Chain de Roteiros
query_roteiro = "Quero visitar os principais pontos turísticos do Rio de Janeiro"

print(f"🔍 **TESTE - CHAIN ROTEIROS:**\n'{query_roteiro}'\n")

# Obter contexto via RAG
contexto = obter_contexto_rag(query_roteiro, "roteiro")

# ✅ SINTAXE MODERNA: Usar invoke()
resposta = chain_roteiro.invoke({
    "contexto": contexto,
    "query": query_roteiro
})

# Extrair conteúdo da resposta
if hasattr(resposta, 'content'):
    print(resposta.content)
else:
    print(resposta)

🔍 **TESTE - CHAIN ROTEIROS:**
'Quero visitar os principais pontos turísticos do Rio de Janeiro'

**Roteiro Turístico Personalizado para o Rio de Janeiro!**

Olá, amigo! Estou aqui para ajudá-lo a criar uma experiência inesquecível no Rio de Janeiro. Com base nos principais pontos turísticos da cidade, vou criar um roteiro personalizado para você.

**Dia 1: Cristo Redentor e Pão de Açúcar**

Comece o dia cedo, com uma visita ao **Cristo Redentor**, uma das Sete Maravilhas do Mundo Moderno. Localizado no Corcovado, a 700 metros de altitude, você terá uma vista panorâmica incrível da cidade. O melhor horário para visitar é às 8h da manhã, antes do calor e da multidão.

Depois de visitar o Cristo, vamos para o **Pão de Açúcar**, um dos bondinhos mais famosos do mundo. Com uma vista incrível da cidade e das praias, você não pode perder essa oportunidade. O melhor horário para visitar é às 10h da manhã, quando o sol está alto e a vista é ainda mais impressionante.

**Dia 1: Praia de Copacaba

## 4️⃣ **Chain 2: Especialista em Logística**

In [18]:
# Template para Chain de Logística
template_logistica = """
🚗 ESPECIALISTA EM LOGÍSTICA DE VIAGEM

Você é um consultor especializado em logística e planejamento de viagens.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Transporte (aeroportos, metro, ônibus, táxi)
- Hospedagem (localização, custo-benefício)
- Locomoção entre pontos turísticos
- Documentação e requisitos de viagem
- Estimativas de tempo e custo

INSTRUÇÕES:
- Foque em aspectos práticos e operacionais
- Forneça informações precisas sobre transporte
- Sugira a melhor forma de chegar aos destinos
- Inclua dicas de economia e eficiência
- Use apenas informações do contexto

RESPOSTA:
"""

# Criar PromptTemplate
prompt_logistica = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_logistica
)

# ✅ SINTAXE MODERNA: Usar RunnableSequence
chain_logistica = prompt_logistica | llm

print("🚗 Chain de Logística criada com sintaxe moderna!")

🚗 Chain de Logística criada com sintaxe moderna!


### **Teste da Chain Logística**

In [19]:
# Teste da Chain de Logística
query_logistica = "Como me locomover em Paris e onde me hospedar?"

print(f"🔍 **TESTE - CHAIN LOGÍSTICA:**\n'{query_logistica}'\n")

# Obter contexto via RAG
contexto = obter_contexto_rag(query_logistica, "logistica")

# ✅ SINTAXE MODERNA: Usar invoke()
resposta = chain_logistica.invoke({
    "contexto": contexto,
    "query": query_logistica
})

# Extrair conteúdo da resposta
if hasattr(resposta, 'content'):
    print(resposta.content)
else:
    print(resposta)

🔍 **TESTE - CHAIN LOGÍSTICA:**
'Como me locomover em Paris e onde me hospedar?'

Olá! Como especialista em logística de viagem, estou aqui para ajudá-lo a planejar sua viagem a Paris de forma eficiente e econômica.

**Transporte**

Para chegar ao aeroporto Charles de Gaulle, recomendo usar o RER B, que é uma linha de trem que conecta o aeroporto ao centro de Paris em cerca de 45 minutos. Você pode comprar um bilhete de trem na estação do aeroporto ou na plataforma do RER B.

Uma vez no centro de Paris, o metro é a melhor opção para se locomover. A cidade tem 14 linhas de metro, e você pode usar o passe Navigo para viajar ilimitadamente durante um dia. O metro funciona até 1h15 da manhã, de segunda a sexta-feira, e até 2h15 da manhã nos fins de semana.

Se você preferir usar um táxi ou Uber, é uma opção disponível, mas é mais rápido e econômico usar o metro no centro da cidade.

**Hospedagem**

O Hotel Le Marais é uma ótima opção para se hospedar em Paris. Localizado no bairro histórico

## 5️⃣ **Chain 3: Especialista em Informações Locais**

In [20]:
# Template para Chain de Informações Locais
template_info_local = """
📍 ESPECIALISTA EM INFORMAÇÕES LOCAIS

Você é um local experiente que conhece todos os segredos da cidade.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Cultura local e costumes
- Gastronomia típica e restaurantes
- Eventos e festivais locais
- Dicas de segurança
- Lugares menos conhecidos mas incríveis
- Horários de funcionamento e feriados locais

INSTRUÇÕES:
- Forneça insights culturais únicos
- Inclua dicas que só um local saberia
- Seja caloroso e acolhedor
- Mencione aspectos culturais importantes
- Use informações do contexto quando disponível
- Se o contexto for limitado, use conhecimento geral respeitoso

RESPOSTA:
"""

# Criar PromptTemplate
prompt_info_local = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_info_local
)

# ✅ SINTAXE MODERNA: Usar RunnableSequence
chain_info_local = prompt_info_local | llm

print("📍 Chain de Informações Locais criada com sintaxe moderna!")

📍 Chain de Informações Locais criada com sintaxe moderna!


### **Teste da Chain Informações Locais**

In [21]:
# Teste da Chain de Informações Locais
query_info = "Quais são os costumes locais que devo conhecer?"

print(f"🔍 **TESTE - CHAIN INFO LOCAIS:**\n'{query_info}'\n")

# Obter contexto via RAG (limitado para este tipo)
contexto = obter_contexto_rag(query_info, "info-local")

# ✅ SINTAXE MODERNA: Usar invoke()
resposta = chain_info_local.invoke({
    "contexto": contexto,
    "query": query_info
})

# Extrair conteúdo da resposta
if hasattr(resposta, 'content'):
    print(resposta.content)
else:
    print(resposta)

🔍 **TESTE - CHAIN INFO LOCAIS:**
'Quais são os costumes locais que devo conhecer?'

Bem-vindo à Paris, a cidade da luz e da beleza! Estou aqui para compartilhar com você os segredos e costumes locais que você precisa conhecer para ter uma experiência autêntica em Paris.

**Costumes locais que você deve conhecer:**

1. **O respeito pela hora do jantar**: Os parisienses são conhecidos por serem pontuais, especialmente quando se trata de jantar. É comum que as pessoas se reúnam às 20h30 para uma refeição prolongada, então certifique-se de chegar no horário.
2. **O amor pela vida noturna**: Paris é famosa por sua vida noturna vibrante. Os bares e clubes estão abertos até tarde da noite, então não hesite em explorar a cidade à noite.
3. **O respeito pela privacidade**: Os parisienses são conhecidos por serem reservados e respeitosos da privacidade. Evite fazer perguntas pessoais ou invadir o espaço de alguém.
4. **O amor pela arte e a cultura**: Paris é a cidade da arte e da cultura. Visite

## 6️⃣ **Chain 4: Especialista em Tradução**

In [22]:
# Template para Chain de Tradução
template_traducao = """
🌐 ESPECIALISTA EM TRADUÇÃO E COMUNICAÇÃO

Você é um tradutor especializado em turismo e comunicação intercultural.

CONTEXTO RELEVANTE:
{contexto}

CONSULTA DO USUÁRIO: {query}

SUAS ESPECIALIDADES:
- Tradução precisa e contextual
- Frases úteis para viajantes
- Expressões locais e gírias
- Comunicação em emergências
- Etiqueta cultural na comunicação
- Pronúncia simplificada

INSTRUÇÕES:
- Forneça traduções precisas e naturais
- Inclua contexto cultural quando relevante
- Adicione dicas de pronúncia quando possível
- Explique nuances culturais da comunicação
- Seja prático e útil para situações reais

RESPOSTA:
"""

# Criar PromptTemplate
prompt_traducao = PromptTemplate(
    input_variables=["contexto", "query"],
    template=template_traducao
)

# ✅ SINTAXE MODERNA: Usar RunnableSequence
chain_traducao = prompt_traducao | llm

print("🌐 Chain de Tradução criada com sintaxe moderna!")

🌐 Chain de Tradução criada com sintaxe moderna!


### **Teste da Chain Tradução**

In [23]:
# Teste da Chain de Tradução
query_traducao = "Como dizer 'onde fica o hotel' em francês?"

print(f"🔍 **TESTE - CHAIN TRADUÇÃO:**\n'{query_traducao}'\n")

# Obter contexto via RAG
contexto = obter_contexto_rag(query_traducao, "traducao")

# ✅ SINTAXE MODERNA: Usar invoke()
resposta = chain_traducao.invoke({
    "contexto": contexto,
    "query": query_traducao
})

# Extrair conteúdo da resposta
if hasattr(resposta, 'content'):
    print(resposta.content)
else:
    print(resposta)

🔍 **TESTE - CHAIN TRADUÇÃO:**
'Como dizer 'onde fica o hotel' em francês?'

Olá! Como especialista em tradução e comunicação, estou aqui para ajudar.

A pergunta do usuário é simples, mas é sempre útil ter uma resposta precisa e contextualizada. Em francês, a pergunta "onde fica o hotel" pode ser traduzida de várias maneiras, dependendo do contexto e da forma como você está se dirigindo ao outro.

Aqui estão algumas opções:

* "Où est l'hôtel?" (pronúncia: "oo eh leh-oh-tel") - Essa é a forma mais comum e direta de perguntar onde fica o hotel.
* "Pouvez-vous me dire où est l'hôtel?" (pronúncia: "poo-vay voo may dehr oo eh leh-oh-tel") - Essa é uma forma mais educada e respeitosa de perguntar, usando a palavra "pouvez-vous" (você pode) e "me dire" (me dizer).
* "Je cherche l'hôtel, pouvez-vous m'aider?" (pronúncia: "zhuh shair luh-oh-tel, pooo-vay voo may dah-yay") - Essa é uma forma mais completa de perguntar, incluindo a ideia de que você está procurando o hotel e precisa de ajuda.

E

## 7️⃣ **Dicionário de Chains**

In [24]:
# Organizar todas as chains em um dicionário
chains_especializadas = {
    'roteiro': {
        'chain': chain_roteiro,
        'nome': 'Especialista em Roteiros',
        'icone': '🗺️',
        'descricao': 'Pontos turísticos, atrações e roteiros personalizados'
    },
    'logistica': {
        'chain': chain_logistica,
        'nome': 'Especialista em Logística',
        'icone': '🚗',
        'descricao': 'Transporte, hospedagem e planejamento prático'
    },
    'info-local': {
        'chain': chain_info_local,
        'nome': 'Especialista em Informações Locais',
        'icone': '📍',
        'descricao': 'Cultura local, costumes e dicas de insider'
    },
    'traducao': {
        'chain': chain_traducao,
        'nome': 'Especialista em Tradução',
        'icone': '🌐',
        'descricao': 'Tradução e comunicação intercultural'
    }
}

print("✅ **CHAINS ESPECIALIZADAS ORGANIZADAS:**\n")
for key, info in chains_especializadas.items():
    print(f"{info['icone']} **{info['nome']}**")
    print(f"   📋 {info['descricao']}")
    print(f"   🔑 Chave: '{key}'\n")

✅ **CHAINS ESPECIALIZADAS ORGANIZADAS:**

🗺️ **Especialista em Roteiros**
   📋 Pontos turísticos, atrações e roteiros personalizados
   🔑 Chave: 'roteiro'

🚗 **Especialista em Logística**
   📋 Transporte, hospedagem e planejamento prático
   🔑 Chave: 'logistica'

📍 **Especialista em Informações Locais**
   📋 Cultura local, costumes e dicas de insider
   🔑 Chave: 'info-local'

🌐 **Especialista em Tradução**
   📋 Tradução e comunicação intercultural
   🔑 Chave: 'traducao'



## 8️⃣ **Função Utilitária para Executar Chains**

In [25]:
def executar_chain_especializada(tipo_chain: str, query: str) -> str:
    """
    Executa uma chain especializada com contexto RAG.
    
    Args:
        tipo_chain: Tipo da chain (roteiro, logistica, info-local, traducao)
        query: Pergunta do usuário
    
    Returns:
        Resposta da chain especializada
    """
    if tipo_chain not in chains_especializadas:
        return f"❌ Chain '{tipo_chain}' não encontrada. Disponíveis: {list(chains_especializadas.keys())}"
    
    try:
        # Obter contexto via RAG
        contexto = obter_contexto_rag(query, tipo_chain)
        
        # ✅ SINTAXE MODERNA: Usar invoke()
        chain_info = chains_especializadas[tipo_chain]
        resposta = chain_info['chain'].invoke({
            "contexto": contexto,
            "query": query
        })
        
        # Extrair conteúdo da resposta
        if hasattr(resposta, 'content'):
            return resposta.content
        else:
            return str(resposta)
        
    except Exception as e:
        return f"❌ Erro na execução: {e}"

print("✅ Função utilitária atualizada para sintaxe moderna!")

✅ Função utilitária atualizada para sintaxe moderna!


### **Teste da Função Utilitária**

In [26]:
# Teste da função utilitária
testes = [
    ("roteiro", "Principais atrações de Paris"),
    ("logistica", "Como ir do aeroporto ao centro do Rio"),
    ("info-local", "Dicas de segurança para turistas"),
    ("traducao", "Como pedir ajuda em inglês")
]

for tipo, pergunta in testes:
    print(f"🎯 **{chains_especializadas[tipo]['icone']} {tipo.upper()}:**")
    print(f"❓ {pergunta}\n")
    
    resposta = executar_chain_especializada(tipo, pergunta)
    print(resposta)
    print("\n" + "="*80 + "\n")

🎯 **🗺️ ROTEIRO:**
❓ Principais atrações de Paris



**Descubra a Paris dos Sonhos!**

Olá, viajante! Estou aqui para ajudá-lo a criar um roteiro turístico personalizado em Paris, a cidade dos sonhos. Aqui estão as principais atrações que você não pode perder:

**Ponto 1: Torre Eiffel (Ilha da Cité)**

Comece sua jornada pela Torre Eiffel, o símbolo de Paris. Localizada na Ilha da Cité, essa estrutura de 324m de altura é uma visão incrível durante o dia e ainda mais impressionante à noite, quando é iluminada de forma espetacular. Não perca a oportunidade de subir ao topo e desfrutar de uma vista panorâmica incrível da cidade.

**Dica:** Visite a Torre Eiffel no final da tarde ou no início da noite para aproveitar a iluminação noturna.

**Ponto 2: Museu do Louvre (Rue de Rivoli)**

Depois de visitar a Torre Eiffel, dirija-se ao Museu do Louvre, localizado na Rue de Rivoli. Com mais de 550.000 obras de arte, é o maior museu do mundo. Não perca a oportunidade de ver a famosa Mona Lisa, além de outras obras-primas da arte clássica.

**Dica:*

## ✅ **Resumo do Notebook 03:**

### 🎯 **Chains Implementadas:**
1. 🗺️ **Chain Roteiros**: Especialista em pontos turísticos e roteiros
2. 🚗 **Chain Logística**: Expert em transporte e hospedagem
3. 📍 **Chain Info Local**: Conhecedor de cultura e costumes locais
4. 🌐 **Chain Tradução**: Tradutor especializado em turismo

### 🛠️ **Funcionalidades:**
- ✅ Templates especializados por domínio
- ✅ Integração com sistema RAG
- ✅ Função utilitária para execução
- ✅ Dicionário organizado de chains

### 📊 **Características Técnicas:**
- 🧠 **LLM**: Groq llama-3.1-8b-instant
- ⚡ **Velocidade**: Resposta rápida (~2s)
- 🎯 **Precisão**: Alta especialização por domínio
- 🔧 **Flexibilidade**: Fácil adição de novas chains

### 🧪 **Testes Realizados:**
- ✅ Todas as 4 chains funcionais
- ✅ Integração RAG operacional
- ✅ Função utilitária testada
- ✅ Respostas especializadas por domínio

### ➡️ **Próximo Passo:**
**04-Router.ipynb** → Sistema de roteamento inteligente

---
⛓️ **4 Chains especializadas prontas para uso!**